# **Shark Attack Dataset Cleaning and Analysis**

In [1]:
import pandas as pd
import numpy as np
import re

In [2]:
attacks = pd.read_csv('attacks.csv', encoding='latin1')

In [3]:
attacks.dropna(how='all', inplace=True) # Drop rows that all values are missing 
attacks.rename({'Sex ': 'Sex', 'Species ': 'Species', 'Fatal (Y/N)': 'Fatal'}, axis='columns', inplace=True)

In [4]:
df = attacks[['Unnamed: 22', 'Unnamed: 23']]
display(df[df['Unnamed: 22'].notnull() | df['Unnamed: 23'].notnull()])
attacks.drop(['Unnamed: 22', 'Unnamed: 23'], axis=1, inplace=True) 
#Drop columns that doesn't have any useful information

,Unnamed: 22,Unnamed: 23
1478,stopped here,NaN
4415,NaN,Teramo
5840,NaN,change filename


In [5]:
attacks.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8703 entries, 0 to 25722
Data columns (total 22 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Case Number             8702 non-null   object 
 1   Date                    6302 non-null   object 
 2   Year                    6300 non-null   float64
 3   Type                    6298 non-null   object 
 4   Country                 6252 non-null   object 
 5   Area                    5847 non-null   object 
 6   Location                5762 non-null   object 
 7   Activity                5758 non-null   object 
 8   Name                    6092 non-null   object 
 9   Sex                     5737 non-null   object 
 10  Age                     3471 non-null   object 
 11  Injury                  6274 non-null   object 
 12  Fatal                   5763 non-null   object 
 13  Time                    2948 non-null   object 
 14  Species                 3464 non-null  

In [6]:
#cleaning column Sex

attacks['Sex'] = attacks['Sex'].str.strip().replace(['lli', 'N', '.'], float('NaN'))
# replacing any unusable information for 'NaN'
attacks['Sex'].unique()

array(['F', 'M', nan], dtype=object)

In [7]:
#cleaning column Age 
 
def ages(x):
    '''
    
    Find all numeric values and clean them by transforming all into int type and store then in a variable.
    Calculate the mean if the object have 2 valid numeric values.
   
    ''' 
    
    age_list = re.findall('\d+.?\d+', x.replace(',','.'))
    age_list_int = [int(float(x)) for x in age_list]
    try:
        return sum(age_list_int)/len(age_list_int)
    except: 
        return np.nan

attacks['Age'] = attacks['Age'].apply(lambda x: ages(str(x))) #apply the ages function on the Age column
attacks['Age'].unique()

array([57. , 11. , 48. ,  nan, 18. , 52. , 15. , 12. , 32. , 10. , 21. ,
       34. , 30. , 60. , 33. , 29. , 54. , 41. , 37. , 56. , 19. , 25. ,
       69. , 38. , 55. , 35. , 46. , 45. , 14. , 40. , 28. , 20. , 24. ,
       26. , 49. , 22. , 31. , 17. , 13. , 42. , 50. , 16. , 82. , 73. ,
       68. , 51. , 39. , 58. , 47. , 61. , 65. , 36. , 66. , 43. , 72. ,
       59. , 27. , 64. , 23. , 71. , 44. , 62. , 63. , 70. , 53. , 77. ,
       74. , 86. , 12.5, 84. , 75. , 21.5, 87. , 67. , 23.5, 24.5, 26.5,
       38.6, 15.5, 29.5, 81. , 78. , 16.5, 13.5])

In [8]:
#Cleaning Columns Type by replacing any unsual types

attacks['Type'] = attacks['Type'].str.strip().replace(['Invalid', 'Questionable'], float('NaN'))
attacks['Type'] = attacks['Type'].str.strip().replace(['Boatomg', 'Boating'], str('Boat'))
attacks['Type'].unique()

array(['Boat', 'Unprovoked', nan, 'Provoked', 'Sea Disaster'],
      dtype=object)

In [9]:
#Cleaning Column Fatal bt replacing any name other than Y, N and NaN

attacks['Fatal'] = attacks['Fatal'].str.strip().replace(['UNKNOWN', '2017', 'M'], float('NaN'))
attacks['Fatal'] = attacks['Fatal'].str.strip().replace(['y'], str('Y'))
attacks['Fatal'].unique()

array(['N', 'Y', nan], dtype=object)

# *Analysing Data*

In [10]:
#Create a Data Frame that shows the number of attacks per Sex

attacks_sex = attacks['Sex'].value_counts()
attacks_sex = pd.DataFrame(attacks_sex)
attacks_sex = attacks_sex.rename(columns={'Sex':'Attacks'}).reset_index()
attacks_sex.head()

,index,Attacks
0,M,5096
1,F,637


In [11]:
#Create a Data Frame that shows the number of Fatal attacks per Sex and Type

attacks_fatal = attacks[['Sex', 'Type', 'Fatal']].groupby(by=['Type', 'Sex']).count().sort_values(by='Fatal' , ascending= False).reset_index()

In [12]:
attacks_fatal.query("Type == 'Unprovoked'")

,Type,Sex,Fatal
0,Unprovoked,M,3860
1,Unprovoked,F,531


In [13]:
attacks_fatal.query("Type == 'Provoked'")

,Type,Sex,Fatal
2,Provoked,M,488
5,Provoked,F,22


In [14]:
# Assign the porcentage of provoked fatal cases per sex to variables and 
# verifying which gender has the higher percentage

fem_prov = 637/2200 
masc_prov =  5096/48800

if fem_prov > masc_prov:
    print('Women are more involved in deaths for provoke sharks.')
else:
     print('Men are more involved in deaths for provoke sharks')

Women are more involved in deaths for provoke sharks.


In [15]:
#Create a Data Frame that shows the number of attacks per Age

attacks_age = attacks['Age'].value_counts()
attacks_age = pd.DataFrame(attacks_age)
attacks_age = attacks_age.rename(columns={'Age':'Population'}).reset_index()
attacks_age.head(20)

,index,Population
0,17.0,155
1,20.0,152
2,18.0,151
3,19.0,144
4,15.0,139
5,16.0,138
6,21.0,120
7,22.0,117
8,25.0,108
9,24.0,106


In [16]:
attacks_age['index'].mean() # mean of all the attacked people's age

43.39879518072289

In [17]:
#Create a Data Frame that shows the number of Fatal attacks per Sex and Age.

attacks_age_fatal = attacks[['Sex', 'Age', 'Fatal']].groupby(by=['Age', 'Sex']).count().sort_values(by='Fatal' , ascending= False).reset_index()

In [18]:
# Divied the Data Frame above per gender 

maskm = (attacks_age_fatal['Sex'] == 'M')
masc_age = attacks_age_fatal[maskm]
masc_age.head(12)

,Age,Sex,Fatal
0,18.0,M,125
1,20.0,M,124
2,17.0,M,124
3,16.0,M,123
4,19.0,M,120
5,15.0,M,115
6,25.0,M,98
7,22.0,M,97
8,21.0,M,97
9,24.0,M,91


In [19]:
masc_age['Age'].mean() # mean of all the attacked men's age

43.72682926829268

In [20]:
maskf = (attacks_age_fatal['Sex'] == 'F')
fem_age = attacks_age_fatal[maskf]
fem_age.head(9)

,Age,Sex,Fatal
36,13.0,F,26
42,14.0,F,22
45,15.0,F,20
46,20.0,F,17
47,10.0,F,16
48,18.0,F,15
49,19.0,F,15
51,21.0,F,14
52,12.0,F,13


In [21]:
fem_age['Age'].mean() # mean of all the attacked women's age

40.34426229508197

In [22]:
attacks.to_csv('exported_attacks.csv', index=False)

In [23]:
pd.read_csv('exported_attacks.csv')

,Case Number,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,...,Fatal,Time,Species,Investigator or Source,pdf,href formula,href,Case Number.1,Case Number.2,original order
0,2018.06.25,25-Jun-2018,2018.0,Boat,USA,California,"Oceanside, San Diego County",Paddling,Julie Wolfe,F,...,N,18h00,White shark,"R. Collier, GSAF",2018.06.25-Wolfe.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.25,2018.06.25,6303.0
1,2018.06.18,18-Jun-2018,2018.0,Unprovoked,USA,Georgia,"St. Simon Island, Glynn County",Standing,Adyson McNeely,F,...,N,14h00 -15h00,NaN,"K.McMurray, TrackingSharks.com",2018.06.18-McNeely.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.18,2018.06.18,6302.0
2,2018.06.09,09-Jun-2018,2018.0,NaN,USA,Hawaii,"Habush, Oahu",Surfing,John Denges,M,...,N,07h45,NaN,"K.McMurray, TrackingSharks.com",2018.06.09-Denges.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.09,2018.06.09,6301.0
3,2018.06.08,08-Jun-2018,2018.0,Unprovoked,AUSTRALIA,New South Wales,Arrawarra Headland,Surfing,male,M,...,N,NaN,2 m shark,"B. Myatt, GSAF",2018.06.08-Arrawarra.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.08,2018.06.08,6300.0
4,2018.06.04,04-Jun-2018,2018.0,Provoked,MEXICO,Colima,La Ticla,Free diving,Gustavo Ramos,M,...,N,NaN,"Tiger shark, 3m",A .Kipper,2018.06.04-Ramos.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.04,2018.06.04,6299.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8698,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8699,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8700,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8701,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
